In [ ]:
#1 без категориальных столбцов
# Стандартизация
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
x_scale = scaler.fit_transform(df_clusters)
df_scale = pd.DataFrame(x_scale, columns=df_clusters.columns)

In [ ]:
# K-means
from sklearn.cluster import KMeans
model = KMeans(n_clusters=3)
y_pred = model.fit_predict(df_scale)
pd.DataFrame(y_pred)[0].value_counts()
# Добавляем обратно категориальные столбцы 
df['y_pred'] = y_pred
df_scale['location'] = df['location']
df_scale['date'] = df['date']
df_scale['continent'] = df['continent']
df_scale['y_pred'] = y_pred

In [ ]:
### Прежде чем применить алгоритм кластеризации, мы должны определить соответствующий уровень эпсилона, используя “Метод локтя”
# DBScan
from sklearn.neighbors import NearestNeighbors
plt.figure(figsize=(10,5))
nn = NearestNeighbors(n_neighbors=10).fit(pca_df)
distances, idx = nn.kneighbors(pca_df)
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.plot(distances)
plt.show()

### Оптимальное значение эпсилона составляет около 0,5. 
#Наконец, поскольку у нас есть 10 основных компонента наших данных, мы установим наши минимальные критерии баллов равными 20.
y_pred = DBSCAN(eps = 0.5, min_samples = 20).fit_predict(pca_df)
import sklearn.metrics as metrics
%%time

db = DBSCAN(eps=0.5, min_samples=20).fit(pca_df)
labels = db.labels_
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print('Предполагаемое число кластеров: %d' % n_clusters_)
print('Расчетное количество точек шума: %d' % n_noise_)
#print("Коэффициент силуэта: %0.3f" % metrics.silhouette_score(pca_df, labels))

### У нас получилось слишком много класстеров попробуем увеличить значения eps и min_samples

%%time
db = DBSCAN(eps=1.0, min_samples=20).fit(pca_df)
labels = db.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print('Предполагаемое число кластеров: %d' % n_clusters_)
print('Расчетное количество точек шума: %d' % n_noise_)

%%time
db = DBSCAN(eps=2, min_samples=10).fit(pca_df)
labels = db.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print('Предполагаемое число кластеров: %d' % n_clusters_)
print('Расчетное количество точек шума: %d' % n_noise_)
# 3мин 7 кластеров

df2["dbscan_lable"] = y_pred

#график


In [ ]:
### из-за невозможности указать количество кластеров в DBSCAN получить желаемый результат сложно, попробуем Birch
from sklearn.cluster import Birch
brc = Birch(n_clusters=3)

%%time
brc.fit(pca_df)
y_pred = brc.predict(pca_df)
pd.DataFrame(y_pred)[0].value_counts()

df2["birch_lable"] = y_pred

#график

In [ ]:
### Лучше всего нам подходят метки knn_lable. Оставим их, удалив остальные
df2=df2.drop(["dbscan_lable","birch_lable"], axis=1)

In [ ]:
### Посмотрим зависимость целевой функции от других атрибутов 
df2.corr()

In [ ]:
# Сохранение файла
df2.to_csv('C:\\Users\\User\\Desktop\\WS\\koronaV.csv', index=False)

In [ ]:
### Классификация
train = train.drop(['Date','continent','location'], axis = 1)
test = test.drop(['Date','continent','location'], axis = 1)

X_train = train.drop(['knn_lable'],axis=1)
X_test = test.drop(['knn_lable'],axis=1)
y_train = train['knn_lable']
y_test = test['knn_lable']

In [ ]:
### Начнем с простого и быстрого KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

%%time
first_knn = KNeighborsClassifier()
knn_params = {'n_neighbors':range(1,10,1)}
knn_grid = GridSearchCV(first_knn,knn_params,cv=5)
knn_grid.fit(X_train,y_train)
print(knn_grid.best_score_)
print(knn_grid.best_params_) 
knn_grid.score(X_test, y_test)



In [ ]:
### Точность 0.965% очень хорошая для такого простого метода, посмотрим что покажут более сложные модели
from sklearn.ensemble import GradientBoostingClassifier

%%time
gbrt = GradientBoostingClassifier(max_depth=5)
gbrt.fit(X_train,y_train)
gbrt.score(X_test, y_test)

%%time
gbrt = GradientBoostingClassifier(max_depth=7)
gbrt.fit(X_train,y_train)
gbrt.score(X_test, y_test)

%%time
gbrt = GradientBoostingClassifier(max_depth=2)
gbrt.fit(X_train,y_train)
gbrt.score(X_test, y_test)

In [ ]:
### Мы получили точность 0.977% что является идеальным, попробуем еще один метод
from sklearn.svm import SVC
svm = SVC(gamma=1, C=1)
svm.fit(X_train,y_train)
svm.score(X_test, y_test)



In [ ]:
# Категориальные
value_counts().max()

# Не категориальные
mean()

In [ ]:
# графики

# Попарные зависимости признаков
cols = ['Age', 'Fare', 'Pclass', 'Sex', 'SibSp','Parch','Embarked','Survived']
sns.pairplot(d2[cols],height=2);

# гистрограмма распределения
d2.hist(figsize = (15,10), bins = 50);

# Зависимость выживания от пола(0 - Ж, 1 -М)
sns.countplot(x='Sex',hue='Survived',data=d2);

# график рассеяния 
plt.scatter(x=d2['Age'],y=d2['Fare'], c=d2['Survived'].map({0: 'blue', 1: 'red'}));

# график рассеяния 
cmap = {0 : "blue", 1: "red"}
sns.scatterplot(data["Age"], data["Fare"], hue = data["Survived"], palette=cmap)
plt.grid()

# кореляционная матрица
sns.heatmap(d3.corr(), annot = True,fmt='.1g',vmin=-1, vmax=1, center= 0);

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)